# EDA & Preprocessing
## Backorer Prediction from Kaggle

## Context

Part backorders is a common supply chain problem. Working to identify parts at risk of backorder before the event occurs so the business has time to react.

### Content

Training data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

sku - Random ID for the product

national_inv - Current inventory level for the part

lead_time - Transit time for product (if available)

in_transit_qty - Amount of product in transit from source

forecast_3_month - Forecast sales for the next 3 months

forecast_6_month - Forecast sales for the next 6 months

forecast_9_month - Forecast sales for the next 9 months

sales_1_month - Sales quantity for the prior 1 month time period

sales_3_month - Sales quantity for the prior 3 month time period

sales_6_month - Sales quantity for the prior 6 month time period

sales_9_month - Sales quantity for the prior 9 month time period

min_bank - Minimum recommend amount to stock

potential_issue - Source issue for part identified

pieces_past_due - Parts overdue from source

perf_6_month_avg - Source performance for prior 6 month period

perf_12_month_avg - Source performance for prior 12 month period

local_bo_qty - Amount of stock orders overdue

deck_risk - Part risk flag

oe_constraint - Part risk flag

ppap_risk - Part risk flag

stop_auto_buy - Part risk flag

rev_stop - Part risk flag

went_on_backorder - Product actually went on backorder. This is the target value.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

C:\Python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load data

In [2]:
train = pd.read_csv("./data/Kaggle_Training_Dataset_v2.csv")
test = pd.read_csv("./data/Kaggle_Test_Dataset_v2.csv")

C:\Python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 23 columns):
sku                  1687861 non-null object
national_inv         1687860 non-null float32
lead_time            1586967 non-null float32
in_transit_qty       1687860 non-null float32
forecast_3_month     1687860 non-null float32
forecast_6_month     1687860 non-null float32
forecast_9_month     1687860 non-null float32
sales_1_month        1687860 non-null float32
sales_3_month        1687860 non-null float32
sales_6_month        1687860 non-null float32
sales_9_month        1687860 non-null float32
min_bank             1687860 non-null float32
potential_issue      1687860 non-null object
pieces_past_due      1687860 non-null float32
perf_6_month_avg     1687860 non-null float32
perf_12_month_avg    1687860 non-null float32
local_bo_qty         1687860 non-null float32
deck_risk            1687860 non-null object
oe_constraint        1687860 non-null object
ppap_risk        

In [50]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Data columns (total 23 columns):
sku                  242076 non-null object
national_inv         242075 non-null float32
lead_time            227351 non-null float32
in_transit_qty       242075 non-null float32
forecast_3_month     242075 non-null float32
forecast_6_month     242075 non-null float32
forecast_9_month     242075 non-null float32
sales_1_month        242075 non-null float32
sales_3_month        242075 non-null float32
sales_6_month        242075 non-null float32
sales_9_month        242075 non-null float32
min_bank             242075 non-null float32
potential_issue      242075 non-null object
pieces_past_due      242075 non-null float32
perf_6_month_avg     242075 non-null float32
perf_12_month_avg    242075 non-null float32
local_bo_qty         242075 non-null float32
deck_risk            242075 non-null object
oe_constraint        242075 non-null object
ppap_risk            242075 non-null o

In [4]:
train.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [5]:
train.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Columns: 23 entries, sku to went_on_backorder
dtypes: float64(15), object(8)
memory usage: 296.2+ MB


In [6]:
test.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Columns: 23 entries, sku to went_on_backorder
dtypes: float64(15), object(8)
memory usage: 42.5+ MB


In [7]:
len(test.sku)/len(train.sku)

0.14342176281103716

In [53]:
train.isnull().sum()

sku                       0
national_inv              1
lead_time            100894
in_transit_qty            1
forecast_3_month          1
forecast_6_month          1
forecast_9_month          1
sales_1_month             1
sales_3_month             1
sales_6_month             1
sales_9_month             1
min_bank                  1
potential_issue           1
pieces_past_due           1
perf_6_month_avg          1
perf_12_month_avg         1
local_bo_qty              1
deck_risk                 1
oe_constraint             1
ppap_risk                 1
stop_auto_buy             1
rev_stop                  1
went_on_backorder         1
dtype: int64

In [54]:
test.isnull().sum()

sku                      0
national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          1
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                1
oe_constraint            1
ppap_risk                1
stop_auto_buy            1
rev_stop                 1
went_on_backorder        1
dtype: int64

### Interim check
- found that dataset is large and only have dtype of float64 and object
        - seperate float columns and object columns
        - may can change into float32
- test set is 14% compare to train set
        - may too small
        - should make validation set from train set
        - maybe 2 datsets for validatation
- found that dataset needs imputation of missing values
        - especially "lead_time"

In [8]:
float_cols = []
object_cols = []

In [9]:
# checking for max float number 
## may can change dtype from float64 to float32 for memory saving

for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        float_cols.append(c)
        print(max(train[c]), '\t', c)
        
print()        
for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        print(max(test[c]), '\t', c)

print()
for c, dtype in zip(test.columns, test.dtypes):
    if dtype == object:
        object_cols.append(c)
        print(c)

12334404.0 	 national_inv
nan 	 lead_time
489408.0 	 in_transit_qty
1427612.0 	 forecast_3_month
2461360.0 	 forecast_6_month
3777304.0 	 forecast_9_month
741774.0 	 sales_1_month
1105478.0 	 sales_3_month
2146625.0 	 sales_6_month
3205172.0 	 sales_9_month
313319.0 	 min_bank
146496.0 	 pieces_past_due
1.0 	 perf_6_month_avg
1.0 	 perf_12_month_avg
12530.0 	 local_bo_qty

12145792.0 	 national_inv
nan 	 lead_time
265272.0 	 in_transit_qty
1510592.0 	 forecast_3_month
2157024.0 	 forecast_6_month
3162260.0 	 forecast_9_month
349620.0 	 sales_1_month
1099852.0 	 sales_3_month
2103389.0 	 sales_6_month
3195211.0 	 sales_9_month
303713.0 	 min_bank
79964.0 	 pieces_past_due
1.0 	 perf_6_month_avg
1.0 	 perf_12_month_avg
6232.0 	 local_bo_qty

sku
potential_issue
deck_risk
oe_constraint
ppap_risk
stop_auto_buy
rev_stop
went_on_backorder


### Convert dtype float64 to float32
- To reduce memory usage, converted data type

In [10]:
for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        train[c] = train[c].astype(np.float32)
train.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Columns: 23 entries, sku to went_on_backorder
dtypes: float32(15), object(8)
memory usage: 199.6+ MB


In [11]:
for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        test[c] = test[c].astype(np.float32)
test.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Columns: 23 entries, sku to went_on_backorder
dtypes: float32(15), object(8)
memory usage: 28.6+ MB


In [12]:
print(float_cols, '\n', object_cols)

['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month', 'forecast_6_month', 'forecast_9_month', 'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank', 'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg', 'local_bo_qty'] 
 ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']


In [24]:
train[float_cols].tail()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
1687856,-1.0,NaN,0.0,5.0,7.0,9.0,1.0,3.0,3.0,8.0,0.0,0.0,-99.00,-99.00,1.0
1687857,-1.0,9.0,0.0,7.0,9.0,11.0,0.0,8.0,11.0,12.0,0.0,0.0,0.86,0.84,1.0
1687858,62.0,9.0,16.0,39.0,87.0,126.0,35.0,63.0,153.0,205.0,12.0,0.0,0.86,0.84,6.0
1687859,19.0,4.0,0.0,0.0,0.0,0.0,2.0,7.0,12.0,20.0,1.0,0.0,0.73,0.78,1.0
1687860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
train[object_cols].tail()

,sku,potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1687856,1373987,No,No,No,No,Yes,No,No
1687857,1524346,No,Yes,No,No,No,No,Yes
1687858,1439563,No,No,No,No,Yes,No,No
1687859,1502009,No,No,No,No,Yes,No,No
1687860,(1687860 rows),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
test[float_cols].tail()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
242071,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.48,0.48,0.0
242072,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.48,0.48,0.0
242073,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.48,0.48,0.0
242074,2913.0,12.0,0.0,0.0,0.0,0.0,0.0,30.0,88.0,88.0,4.0,0.0,0.48,0.48,0.0
242075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
test[object_cols].tail()

,sku,potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
242071,3526988,No,Yes,No,No,Yes,No,No
242072,3526989,No,Yes,No,No,Yes,No,No
242073,3526990,No,Yes,No,No,Yes,No,No
242074,3526991,No,Yes,No,No,Yes,No,No
242075,(242075 rows),NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Convert to binaries
- last data record have all 'NaN' value
- object dtype columns (except "sku") have 'Yes' or 'No'
- look if it okay to convert into '1' and '0'

In [28]:
print(train.info(verbose=False))
print()
print(test.info(verbose=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Columns: 23 entries, sku to went_on_backorder
dtypes: float32(15), object(8)
memory usage: 199.6+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Columns: 23 entries, sku to went_on_backorder
dtypes: float32(15), object(8)
memory usage: 28.6+ MB
None


In [30]:
# sum of value counts be 1687860 in train data
for i in object_cols[1:]:
    print(i)
    print(train[i].value_counts())
    print(train[i].value_counts()[0] + train[i].value_counts()[1])
    print()
    
# sum of value counts be 242075 in test data
for i in object_cols[1:]:
    print(i)
    print(test[i].value_counts())
    print(test[i].value_counts()[0] + test[i].value_counts()[1])
    print()

potential_issue
No     1686953
Yes        907
Name: potential_issue, dtype: int64
1687860

deck_risk
No     1300377
Yes     387483
Name: deck_risk, dtype: int64
1687860

oe_constraint
No     1687615
Yes        245
Name: oe_constraint, dtype: int64
1687860

ppap_risk
No     1484026
Yes     203834
Name: ppap_risk, dtype: int64
1687860

stop_auto_buy
Yes    1626774
No       61086
Name: stop_auto_buy, dtype: int64
1687860

rev_stop
No     1687129
Yes        731
Name: rev_stop, dtype: int64
1687860

went_on_backorder
No     1676567
Yes      11293
Name: went_on_backorder, dtype: int64
1687860

potential_issue
No     241993
Yes        82
Name: potential_issue, dtype: int64
242075

deck_risk
No     194105
Yes     47970
Name: deck_risk, dtype: int64
242075

oe_constraint
No     242028
Yes        47
Name: oe_constraint, dtype: int64
242075

ppap_risk
No     213357
Yes     28718
Name: ppap_risk, dtype: int64
242075

stop_auto_buy
Yes    232617
No       9458
Name: stop_auto_buy, dtype: int64
24207

In [ ]:
from sklearn.model_selection import train_test_split
train_test_split()